In [1]:
import numpy as np

In [2]:
def padding_sequence(seq, max_len = 501, repkey = 'N'):
    seq_len = len(seq)
    if seq_len < max_len:
        gap_len = max_len -seq_len
        new_seq = seq + repkey * gap_len
    else:
        new_seq = seq[:max_len]
    return new_seq

In [3]:
def get_RNA_seq_concolutional_array(seq, motif_len = 4):
    seq = seq.replace('U', 'T')
    alpha = 'ACGT'
    #for seq in seqs:
    #for key, seq in seqs.iteritems():
    row = (len(seq) + 2*motif_len - 2)
    new_array = np.zeros((row, 4))
    for i in range(motif_len-1):
        new_array[i] = np.array([0.25]*4)
    
    for i in range(row-3, row):
        new_array[i] = np.array([0.25]*4)
        
    #pdb.set_trace()
    for i, val in enumerate(seq):
        i = i + motif_len-1
        if val not in 'ACGT':
            new_array[i] = np.array([0.25]*4)
            continue
        #if val == 'N' or i < motif_len or i > len(seq) - motif_len:
        #    new_array[i] = np.array([0.25]*4)
        #else:
        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except:
            pdb.set_trace()
        #data[key] = new_array
    return new_array


In [4]:
def get_bag_data_1_channel(data, max_len = 501):
    bags = []
    seqs = data["seq"]
    labels = data["Y"]
    for seq in seqs:
        #pdb.set_trace()
        #bag_seqs = split_overlap_seq(seq)
        
        # replace each sequence with the sequence followed by N's until the length is equal to max_len
        bag_seq = padding_sequence(seq, max_len = max_len)
        #flat_array = []
        bag_subt = []
        #for bag_seq in bag_seqs:
        
        # turn the padded sequence into a 2-D array, where a row of the array is [.25,.25,.25,.25] for an N, and [1,0,0,0] for an A, [0,1,0,0] for an T, and so on
        tri_fea = get_RNA_seq_concolutional_array(bag_seq)
        bag_subt.append(tri_fea.T)

        
        bags.append(np.array(bag_subt))
    
        
    return bags, labels

In [5]:
def read_seq_graphprot(seq_file, label = 1):
    seq_list = []
    labels = []
    seq = ''
    with open(seq_file, 'r') as fp:
        for line in fp:
            if line[0] == '>':
                name = line[1:-1]
            else:
                seq = line[:-1].upper()
                seq = seq.replace('T', 'U')
                seq_list.append(seq)
                labels.append(label)
    
    return seq_list, labels

In [6]:
def read_data_file(posifile, negafile = None, train = True):
    data = dict()
    seqs, labels = read_seq_graphprot(posifile, label = 1)
    if negafile:
        seqs2, labels2 = read_seq_graphprot(negafile, label = 0)
        seqs = seqs + seqs2
        labels = labels + labels2
    # made a dictionary, "seq" key has a list of sequences as values, "Y" key has a list of 0's and 1's as values inidicating if the corresponding sequence is bound or not
    data["seq"] = seqs 
    data["Y"] = np.array(labels)
    
    return data

In [7]:
def get_data(posi, nega = None, channel = 7,  window_size = 101, train = True):
    data = read_data_file(posi, nega, train = train)
    if channel == 1:
        # pad the sequences and turn them into one-hot-encoded 2D arrays
        train_bags, label = get_bag_data_1_channel(data, max_len = window_size)

    else:
        train_bags, label = get_bag_data(data, channel = channel, window_size = window_size)
    
    return train_bags, label

In [8]:
# TASK 1: data exploration
train_data_dict = read_data_file('./CLIPSEQ_AGO2.train.positives.fa', './CLIPSEQ_AGO2.train.negatives.fa', train=True)
test_data_dict = read_data_file('./CLIPSEQ_AGO2.ls.positives.fa', './CLIPSEQ_AGO2.ls.negatives.fa', train=False)

In [9]:
print("The number of training samples is {} and the number of testing samples is {}".format(len(train_data_dict["seq"]), len(test_data_dict["seq"])))

sum = 0
for seq in train_data_dict["seq"]:
    sum += len(seq)
avg_seq_len = sum/len(train_data_dict["seq"])

sum = 0



print("The average sequence length is {} basepairs".format(avg_seq_len))

pos = train_data_dict["Y"].sum()
pos_test = test_data_dict["Y"].sum()
print("The number of positive training samples is {}, negative {}, positive testing {}, negative testing {}".format(pos, len(train_data_dict["Y"])-pos, pos_test, len(test_data_dict["Y"])-pos_test ))                                                                                                         

The number of training samples is 92346 and the number of testing samples is 1000
The average sequence length is 335.3414333051783 basepairs
The number of positive training samples is 48095, negative 44251, positive testing 500, negative testing 500


In [10]:
# read in the data 
f, l = get_data('./CLIPSEQ_AGO2.train.positives.fa', './CLIPSEQ_AGO2.train.negatives.fa', channel=1)

In [11]:
f_test, l_test = get_data('./CLIPSEQ_AGO2.ls.positives.fa','CLIPSEQ_AGO2.ls.negatives.fa', channel=1, train=False)

In [12]:
f = np.array(f)
f = np.swapaxes(f, -1, 1)

In [13]:
f_test = np.array(f_test)
f_test = np.swapaxes(f_test, -1, 1)

In [18]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [15]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(10, 3),
                 activation='relu',
                 input_shape=f.shape[1:], padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(128, (10, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(256, (5, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(Dropout(0.25))
# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 107, 4, 128)       3968      
_________________________________________________________________
dropout (Dropout)            (None, 107, 4, 128)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 35, 4, 128)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 35, 4, 128)        163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 4, 128)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 4, 128)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 4, 256)        1

2021-11-27 07:17:40.657190: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-27 07:17:40.661666: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-27 07:17:40.680767: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
opt = SGD(lr=0.01)
# model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.compile(loss=keras.losses.binary_crossentropy, optimizer=opt,metrics=['accuracy'])

In [20]:
model.fit(f, l,
          batch_size=10,
          epochs=50,
          verbose=1,
          validation_data=(f_test, l_test))

Epoch 1/50
9235/9235 [==============================] - 431s 47ms/step - loss: 0.6918 - accuracy: 0.5237 - val_loss: 0.6837 - val_accuracy: 0.5910
Epoch 2/50
9235/9235 [==============================] - 430s 47ms/step - loss: 0.6830 - accuracy: 0.5623 - val_loss: 0.6758 - val_accuracy: 0.5970
Epoch 3/50
9235/9235 [==============================] - 431s 47ms/step - loss: 0.6787 - accuracy: 0.5769 - val_loss: 0.6684 - val_accuracy: 0.6100
Epoch 4/50
9235/9235 [==============================] - 429s 46ms/step - loss: 0.6736 - accuracy: 0.5892 - val_loss: 0.6620 - val_accuracy: 0.6200
Epoch 5/50
9235/9235 [==============================] - 430s 47ms/step - loss: 0.6695 - accuracy: 0.5936 - val_loss: 0.6606 - val_accuracy: 0.6120
Epoch 6/50
9235/9235 [==============================] - 432s 47ms/step - loss: 0.6639 - accuracy: 0.6039 - val_loss: 0.6533 - val_accuracy: 0.6250
Epoch 7/50
9235/9235 [==============================] - 430s 47ms/step - loss: 0.6593 - accuracy: 0.6064 - val_loss: 0

KeyboardInterrupt: 